In [ ]:
import fitz

doc = fitz.open(r"C:\Users\Tarun Akash\Downloads\2109.13916v5.pdf")
text = ""
for page in doc:
    text += page.get_text("text")
print(text)


In [ ]:
import re

def clean_text(text):
    text = re.sub(r"\b(page \d+|footnote|Header)\b", "", text)
    text = text.strip()
    return text

cleaned_text = clean_text(text)
print(cleaned_text[:500])

In [ ]:
data = [{"text": cleaned_text}]

from datasets import Dataset

dataset = Dataset.from_dict({"text": [entry["text"] for entry in data]})
print(dataset)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2" 
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token